#### Edit Data Paths
##### Edit input and outputs paths. Input path should direct to the csv that has the sequences to create the embeddings for, and the output path should direct to the csv that will hold the embeddings.

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
cd ..

/home/romi/projects/cvc


In [6]:
# the sequences in the csv file need to be in a column called Sequences
input_path = "/home/romi/projects/cvc/CDR3_data/cdrs_ch.csv"
output_path = "/home/romi/projects/cvc/CDR3_data/TRA_specific/cdrs_ch_embeddings.csv"

#### Set Environment

In [7]:
import os, sys
import pandas as pd
SRC_DIR = "cvc"
assert os.path.isdir(SRC_DIR), f"Cannot find src dir: {SRC_DIR}"
sys.path.append(SRC_DIR)

from cvc import model_utils

from lab_notebooks.utils import SC_TRANSFORMER, TRANSFORMER, DEVICE
MODEL_DIR = os.path.join(SRC_DIR, "models")
sys.path.append(MODEL_DIR)

FILT_EDIT_DIST = True

#### Load Data

In [8]:
# load data
tcrb_data = pd.read_csv(input_path)
tcrb_data = tcrb_data.drop_duplicates()

In [9]:
tcrb_data

,CAVNTGGFKTIF
0,CAVNTGNQFYF
1,CAVNTNAGKSTF
2,CAVNQAGTALIF
3,CAENSGGSNYKLTF
4,CAVNSGGYQKVTF
...,...
1286516,YYGNNRLAF
1286517,YYNNAGNMLTF
1286518,YYTGFQKLVF
1286519,YYYKLSF


In [10]:
# rename sequences column if not named "Sequences"
tcrb_data.rename(columns={'CAVNTGGFKTIF': 'Sequences'}, inplace=True)

In [17]:
# rename sequences column if not named "Sequences"
tcrb_data.rename(columns={'cdr3': 'Sequences'}, inplace=True)

In [11]:
tcrb_data_small = tcrb_data

#### Create embeddings

In [12]:
TRANSFORMER_TO_USE = TRANSFORMER
from cvc.embbeding_wrapper import EmbeddingWrapper

# Create embeddings
embed_wrap = EmbeddingWrapper(TRANSFORMER_TO_USE, DEVICE, tcrb_data_small, batch_size=128, method="mean", layers=[-1], pbar=True, max_len=120)
embed_wrap.embeddings.shape

Some weights of the model checkpoint at /home/romi/projects/cvc/output_5mil_even_priv_pub were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/10051 [00:00<?, ?it/s]

(1286521, 768)

In [13]:
tcrb_embeddings_df = pd.DataFrame(embed_wrap.embeddings)
tcrb_embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.342200,-0.039131,0.024364,0.002047,-0.038388,0.392898,-0.526527,-0.392826,0.031321,-0.459342,...,0.088256,-0.517040,-0.173820,0.931719,0.264418,-0.364445,-0.615103,0.328522,0.463922,-0.009616
1,-0.250027,0.165451,0.108095,-0.015710,0.016609,0.810453,-0.806638,-0.432446,0.049708,-0.001480,...,0.339119,-0.440854,0.237067,0.980542,0.809353,-0.009868,-1.094788,0.317744,0.441669,0.101527
2,0.254921,-0.283284,0.527503,-0.183063,0.106648,0.266672,-0.925886,-0.138216,0.508035,0.117907,...,0.075133,-0.094339,0.690783,0.773075,0.447528,-0.134554,-0.184237,0.730375,0.689972,-0.248459
3,0.073883,0.747197,-0.649398,-0.185154,-0.081860,0.385906,-0.256280,0.031309,0.150673,-0.558873,...,0.175184,-0.801784,0.123028,0.137784,-0.205314,0.269643,-0.437018,0.301621,0.382381,0.282190
4,-0.305922,0.016374,-0.067755,0.090763,0.181161,0.884653,-0.772833,-0.329227,0.324556,-0.313745,...,0.524484,-0.669569,-0.180235,0.360711,0.164014,-0.037375,-0.949427,0.159827,0.984901,-0.121339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286516,0.058717,-0.308579,-0.262099,-0.054859,-0.132515,0.996860,-0.620408,0.547731,0.110018,0.332326,...,0.468424,-1.310545,0.602931,1.337064,0.389003,-0.325747,-0.742911,0.293825,0.608908,-0.311827
1286517,-0.310181,0.218833,-0.381438,0.115090,0.078367,0.920949,-0.919474,0.595742,-0.178429,0.027989,...,0.509814,-1.012692,0.254789,1.290967,0.464704,0.004448,-0.483082,0.112371,0.266507,-0.075346
1286518,-0.413299,-0.294614,-0.018629,0.114556,0.053435,0.224865,-0.705590,-0.080138,-0.219418,-0.021200,...,0.614880,-1.289953,0.848309,0.918448,-0.078723,-0.231170,-0.250629,0.776994,1.152385,-0.628615
1286519,-0.191337,-0.096807,-0.468298,-0.591568,0.086043,0.923582,-1.242533,0.704407,0.025112,-0.199474,...,-0.011658,-1.329002,1.312534,0.577349,-0.179691,0.091589,-0.785138,0.639800,1.233818,-0.385320


In [ ]:
# create anndata object
tcrb_embeddings_adata = embed_wrap.create_anndata()

# add public/private label to dataframe
df_embeddings_with_label = tcrb_embeddings_df
df_embeddings_with_label['Sequences']=list(tcrb_embeddings_adata.obs['Sequences'])

df_embeddings_with_label

In [16]:
# output embeddings to csv
tcrb_embeddings_df.to_csv(output_path)

In [18]:
tcrb_embeddings_df.to_csv("/home/romi/projects/cvc/CDR3_data/TRA_specific/cdrs_ch_embeddings.gz", compression='gzip')